In [385]:
import pandas as pd
import numpy as np
import sklearn as sk
import re

In [386]:
data = pd.read_csv('/home/maxim/DS/Phase_0/Phase_0_final_proj/Phase0-Project/tmp_data/data_for_graph')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23368 entries, 0 to 23367
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                23368 non-null  int64  
 1   ID  объявления            23368 non-null  int64  
 2   Количество комнат         22327 non-null  object 
 3   Тип                       23368 non-null  object 
 4   Метро                     22053 non-null  object 
 5   Адрес                     23368 non-null  object 
 6   Площадь, м2               23368 non-null  object 
 7   Дом                       23368 non-null  object 
 8   Парковка                  9951 non-null   object 
 9   Цена                      23368 non-null  float64
 10  Ремонт                    20613 non-null  object 
 11  Площадь комнат, м2        14458 non-null  object 
 12  Балкон                    15390 non-null  object 
 13  Окна                      16755 non-null  object 
 14  Санузе

# Очистка выбросов по цене методом IQR

In [387]:

Q1 = data['Цена'].quantile(0.25)
Q3 = data['Цена'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

upper_bound

np.float64(132000.0)

In [388]:
data = data[(data['Цена'] >= lower_bound) & (data['Цена'] <= upper_bound)]


# Площадь оставить только общую (М)

In [389]:
def clean_area(area_value):
    if pd.isna(area_value):
        return None
    
    # Преобразуем в строку и берем первую часть до слеша
    area_str = str(area_value).split('/')[0].strip()
    
    # Извлекаем число из строки
    numbers = re.findall(r'\d+\.?\d*', area_str)
    if numbers:
        return float(numbers[0])
    else:
        return None

# Применяем функцию к колонке
data['Площадь, м2'] = data['Площадь, м2'].apply(clean_area)

In [390]:
area_median = data['Площадь, м2'].median()
data['Площадь, м2'] = data['Площадь, м2'].fillna(area_median)

# Отфильтровать значения в столбце количество комнат и привести к формату ИНТ

In [391]:
def intchanger(text):
    try:
        res = int(''.join(list(filter(lambda x: x.isdigit() == True, text))))
        return (res)
    except:
        return np.nan

In [392]:
data['Количество комнат'] = data['Количество комнат'].apply(intchanger)

# Вычислить ремонт по стоимости квартиры   (М)

In [393]:
data[['Цена', 'Ремонт', 'Количество комнат']]

,Цена,Ремонт,Количество комнат
8,130000.0,Евроремонт,3.0
11,100000.0,Евроремонт,3.0
16,120000.0,Евроремонт,2.0
48,90000.0,Евроремонт,2.0
53,120000.0,NaN,2.0
...,...,...,...
23363,42000.0,Евроремонт,NaN
23364,45000.0,Евроремонт,1.0
23365,50000.0,Дизайнерский,2.0
23366,55000.0,Евроремонт,2.0


In [394]:

median_prices = data.groupby(['Количество комнат', 'Ремонт'])['Цена'].median().reset_index()
median_prices.rename(columns={'Цена': 'Медианная_цена'}, inplace=True)

# Слияние основного DataFrame с медианными 
data = data.merge(median_prices, on=['Количество комнат', 'Ремонт'], how='left')

In [395]:
set(median_prices['Ремонт'])

{'Без ремонта', 'Дизайнерский', 'Евроремонт', 'Косметический'}

In [396]:
data = data.merge(median_prices, on=['Количество комнат', 'Ремонт'])
data

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,...,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Залог,Медианная_цена_x,Медианная_цена_y
0,8,273973191,3.0,Квартира,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0,"5/10, Сталинский",открытая,130000.0,...,Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",NaN,NaN,3.00,Пасс (1),Нет,130000.0,75000.0,75000.0
1,11,273999490,3.0,Квартира,м. Смоленская (2 мин пешком),"Москва, Новинский бульвар, 7",80.0,"4/8, Кирпичный",NaN,100000.0,...,NaN,"Мебель в комнатах, Мебель на кухне, Душевая ка...",NaN,NaN,NaN,Пасс (1),NaN,110000.0,75000.0,75000.0
2,16,274995950,2.0,Квартира,м. Арбатская (10 мин пешком),"Москва, улица Новый Арбат, 15",30.0,19/26,NaN,120000.0,...,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,NaN,NaN,NaN,120000.0,55000.0,55000.0
3,48,271265359,2.0,Квартира,м. Смоленская (5 мин пешком),"Москва, улица Арбат, 45/24",60.0,5/6,наземная,90000.0,...,Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.20,Пасс (1),NaN,90000.0,55000.0,55000.0
4,68,274223065,2.0,Квартира,м. Смоленская (8 мин пешком),"Москва, Большой Николопесковский переулок, 3",60.0,3/6,NaN,120000.0,...,NaN,"Мебель в комнатах, Мебель на кухне, Душевая ка...",NaN,NaN,NaN,NaN,NaN,120000.0,75000.0,75000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17095,23361,202336163,1.0,Квартира,м. Солнцево (10 мин пешком),"Москва, Солнцевский проспект, 32, ш. Боровское...",32.0,"9/9, Панельный",NaN,35000.0,...,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Стиральная...",NaN,NaN,2.64,Пасс (1),Да,35000.0,33000.0,33000.0
17096,23364,274654844,1.0,Квартира,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7,"5/18, Монолитный",NaN,45000.0,...,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Лучи,NaN,NaN,"Пасс (1), Груз (1)",NaN,45000.0,40000.0,40000.0
17097,23365,268679909,2.0,Квартира,м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",NaN,50000.0,...,Можно с детьми,"Мебель на кухне, Душевая кабина, Стиральная ма...",NaN,NaN,NaN,NaN,NaN,50000.0,75000.0,75000.0
17098,23366,274807525,2.0,Квартира,м. Солнцево (11 мин пешком),"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",52.5,"8/23, Монолитный",наземная,55000.0,...,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,2.65,Пасс (3),Да,50000.0,55000.0,55000.0


# Заполнить колонку "Можно с детьми" 

In [397]:
data['Можно с детьми/животными'] = data['Можно с детьми/животными'].fillna('Можно с детьми')

In [398]:
set(data['Можно с детьми/животными'])

{'Можно с детьми', 'Можно с детьми, Можно с животными', 'Можно с животными'}

# Удаление колонок не влияющих на корреляцию цены

In [399]:
to_drop = ['Название ЖК', "Unnamed: 0", 'Серия дома','Мусоропровод', 'Площадь комнат, м2']
for col in to_drop:
    if col in data.columns:
        data.drop(columns=[col], axis=1, inplace=True)

# Заполнить столбец лифт пасажирскими лифтами и заполнение других пропущенных значений

In [400]:
def has_missing_values(column):
    return (
        column.isnull().any() or      # Обнаружение стандартных NaN
        column.eq('').any() or        # Пустые строки
        column.str.contains('^\\\\s*$').any() or  # Строки, содержащие только пробелы
        column.str.contains('nan|none|-|n/a', case=False).any()  # Другие возможные обозначения пропусков
    )

# Колонки для обработки
basic_columns = ['Балкон', 'Окна', 'Санузел', 'Лифт', 'Дополнительно', 'Ремонт']

for col in basic_columns:
    if col in data.columns:
        # Если есть пустые значения (любого вида)
        if has_missing_values(data[col]):
            # Приведение всех пропусков к стандартному виду (np.nan)
            data[col] = data[col].replace(['', ' ', 'nan', 'none', '-', 'n/a'], np.nan)
        
            # Определение самого распространённого значения (если оно существует)
            mode_value = data[col].mode()[0] if not data[col].mode().empty else 'Не указано'
            
            # Замена оставшихся пропусков модой
            data[col] = data[col].fillna(mode_value)
   

#  Заполнить залог данными с цены  (М)

In [401]:
data['Залог'] = data['Залог'].replace([None, 'None', 'none', 'nan', 'NaN', 'N/A', ''], np.nan)
data['Залог'] = pd.to_numeric(data['Залог'], errors='coerce')
data['Залог'] = data['Залог'].fillna(data['Цена'])

# Заполнить парковку

In [402]:
data['Парковка'] = data['Парковка'].fillna('наземная')

In [403]:
data['Высота потолков, м'] = data['Высота потолков, м'].fillna(2.5)

In [404]:
def process_metro_columns(df, metro_col='Метро', address_col='Адрес', 
                         station_col='Название станции метро', time_col='Время до метро'):
    
    # Полный список станций московского метро
    moscow_metro_stations = [
        "Смоленская", "Арбатская", "Китай-город", "Охотный ряд", 
        "Тверская", "Пушкинская", "Чеховская", "Трубная", "Лубянка",
        "Кузнецкий мост", "Площадь Революции", "Театральная",
        "Александровский сад", "Библиотека имени Ленина", "Боровицкая",
        "Парк культуры", "Октябрьская", "Добрынинская", "Павелецкая",
        "Таганская", "Курская", "Комсомольская", "Проспект Мира",
        "Новослободская", "Белорусская", "Краснопресненская", "Киевская",
        
    ]
    
    # 1. Функция для извлечения времени пешком
    def extract_walk_time(metro_text):
        if pd.isna(metro_text):
            return ""
        
        metro_str = str(metro_text)
        # Ищем шаблон: (X мин пешком)
        walk_time_pattern = r"\((\d+)\s*мин"
        match = re.search(walk_time_pattern, metro_str)
        
        if match:
            minutes = match.group(1)
            return f"{minutes} мин."
        else:
            return ""
    
    # 2. Функция для извлечения названия станции из колонки "Метро"
    def extract_station_name(metro_text):
        if pd.isna(metro_text):
            return None
        
        metro_str = str(metro_text).strip()
        
        # Паттерны для извлечения названия станции
        patterns = [
            r'м\.\s*([^(]+)',           # м. Смоленская (5 мин)
            r'метро\s*([^(]+)',         # метро Смоленская (5 мин)
            r'ст\.\s*метро\s*([^(]+)',  # ст. метро Смоленская
            r'^([^(]+)\s*\(',           # Смоленская (5 мин)
        ]
        
        for pattern in patterns:
            match = re.search(pattern, metro_str, re.IGNORECASE)
            if match:
                station = match.group(1).strip()
                # Очищаем название
                station = re.sub(r'[,\-\–\—].*$', '', station).strip()
                
                # Проверяем, является ли извлеченное название известной станцией
                for known_station in moscow_metro_stations:
                    if known_station.lower() in station.lower() or station.lower() in known_station.lower():
                        return known_station
                
                return station
        
        # Если не нашли по паттернам, проверяем прямое вхождение станций
        for station in moscow_metro_stations:
            if station.lower() in metro_str.lower():
                return station
        
        return metro_str.split('(')[0].strip() if '(' in metro_str else metro_str
    
    # 3. Функция для извлечения станции метро из адреса
    def extract_station_from_address(address_text):
        if pd.isna(address_text):
            return None
        
        address_str = str(address_text).lower()
        
        # Ищем станции метро в адресе
        for station in moscow_metro_stations:
            if station.lower() in address_str:
                return station
        
        # Ищем по паттернам
        patterns = [
            r'м\.\s*([^,\.\(]+)',
            r'метро\s*([^,\.\(]+)',
            r'ст\.\s*метро\s*([^,\.\(]+)'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, address_str)
            if match:
                station_candidate = match.group(1).strip()
                # Проверяем, является ли найденная станция известной
                for known_station in moscow_metro_stations:
                    if known_station.lower() in station_candidate or station_candidate in known_station.lower():
                        return known_station
                return station_candidate.title()
        
        return None
    
    # Создаем колонку с временем до метро
    data[time_col] = data[metro_col].apply(extract_walk_time)
    
    # Создаем колонку с названием станции
    data[station_col] = data[metro_col].apply(extract_station_name)
    
    # Заполняем пропуски в названии станции из адреса
    mask_missing = data[station_col].isna()
    data.loc[mask_missing, station_col] = data.loc[mask_missing, address_col].apply(extract_station_from_address)
    
    # Заполняем оставшиеся пропуски
    data[station_col] = data[station_col].fillna('Не указано')
    
    # Заменяем исходную колонку "Метро" на время до метро
    data[metro_col] = data[time_col]
    
    # Удаляем временную колонку, если она совпадает с исходной
    if time_col != metro_col:
        df_result = data.drop(columns=[time_col])
    
    return data

# Применяем функцию
data = process_metro_columns(data)

In [405]:
data.head()

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,...,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Залог,Медианная_цена_x,Медианная_цена_y,Время до метро,Название станции метро
0,273973191,3.0,Квартира,9 мин.,"Москва, Новинский бульвар, 18С1",120.0,"5/10, Сталинский",открытая,130000.0,Евроремонт,...,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",3.0,Пасс (1),130000.0,75000.0,75000.0,9 мин.,Смоленская
1,273999490,3.0,Квартира,2 мин.,"Москва, Новинский бульвар, 7",80.0,"4/8, Кирпичный",наземная,100000.0,Евроремонт,...,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Душевая ка...",2.5,Пасс (1),110000.0,75000.0,75000.0,2 мин.,Смоленская
2,274995950,2.0,Квартира,10 мин.,"Москва, улица Новый Арбат, 15",30.0,19/26,наземная,120000.0,Евроремонт,...,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",2.5,Пасс (1),120000.0,55000.0,55000.0,10 мин.,Арбатская
3,271265359,2.0,Квартира,5 мин.,"Москва, улица Арбат, 45/24",60.0,5/6,наземная,90000.0,Евроремонт,...,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),90000.0,55000.0,55000.0,5 мин.,Смоленская
4,274223065,2.0,Квартира,8 мин.,"Москва, Большой Николопесковский переулок, 3",60.0,3/6,наземная,120000.0,Дизайнерский,...,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Душевая ка...",2.5,Пасс (1),120000.0,75000.0,75000.0,8 мин.,Смоленская


In [406]:
data.isnull().sum()

ID  объявления              0
Количество комнат           0
Тип                         0
Метро                       0
Адрес                       0
Площадь, м2                 0
Дом                         0
Парковка                    0
Цена                        0
Ремонт                      0
Балкон                      0
Окна                        0
Санузел                     0
Можно с детьми/животными    0
Дополнительно               0
Высота потолков, м          0
Лифт                        0
Залог                       0
Медианная_цена_x            0
Медианная_цена_y            0
Время до метро              0
Название станции метро      0
dtype: int64

In [407]:
set(data['Ремонт'])

{'Без ремонта', 'Дизайнерский', 'Евроремонт', 'Косметический'}

In [408]:
data.head()

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,...,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Залог,Медианная_цена_x,Медианная_цена_y,Время до метро,Название станции метро
0,273973191,3.0,Квартира,9 мин.,"Москва, Новинский бульвар, 18С1",120.0,"5/10, Сталинский",открытая,130000.0,Евроремонт,...,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",3.0,Пасс (1),130000.0,75000.0,75000.0,9 мин.,Смоленская
1,273999490,3.0,Квартира,2 мин.,"Москва, Новинский бульвар, 7",80.0,"4/8, Кирпичный",наземная,100000.0,Евроремонт,...,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Душевая ка...",2.5,Пасс (1),110000.0,75000.0,75000.0,2 мин.,Смоленская
2,274995950,2.0,Квартира,10 мин.,"Москва, улица Новый Арбат, 15",30.0,19/26,наземная,120000.0,Евроремонт,...,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",2.5,Пасс (1),120000.0,55000.0,55000.0,10 мин.,Арбатская
3,271265359,2.0,Квартира,5 мин.,"Москва, улица Арбат, 45/24",60.0,5/6,наземная,90000.0,Евроремонт,...,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),90000.0,55000.0,55000.0,5 мин.,Смоленская
4,274223065,2.0,Квартира,8 мин.,"Москва, Большой Николопесковский переулок, 3",60.0,3/6,наземная,120000.0,Дизайнерский,...,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Душевая ка...",2.5,Пасс (1),120000.0,75000.0,75000.0,8 мин.,Смоленская


In [409]:
print(data.columns.tolist())

['ID  объявления', 'Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Ремонт', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м', 'Лифт', 'Залог', 'Медианная_цена_x', 'Медианная_цена_y', 'Время до метро', 'Название станции метро']


In [410]:
# Если знаете точный порядок, который нужен
desired_order = [
    'ID  объявления',
    'Количество комнат',
    'Тип',
    'Название станции метро', 
    'Время до метро',
    'Адрес',
    'Площадь, м2',
    'Цена',
    'Залог',
    'Медианная_цена_x',
    'Медианная_цена_y',
    'Лифт',
    'Высота потолков, м',
    'Дополнительно',
    'Можно с детьми/животными',
    'Ремонт',
    'Балкон', 
    'Окна', 
    'Санузел',
    'Дом', 
    'Парковка'
]

# Проверяем, что все колонки существуют
existing_cols = [col for col in desired_order if col in data.columns]
missing_cols = [col for col in desired_order if col not in data.columns]
extra_cols = [col for col in data.columns if col not in desired_order]

# Добавляем колонки, которые не вошли в desired_order, в конец
final_order = existing_cols + extra_cols

data = data[final_order]

if missing_cols:
    print(f"Не найдены колонки: {missing_cols}")

In [411]:
# Текущие русские названия колонок
current_names = [
    'ID  объявления',
    'Количество комнат',
    'Тип',
    'Название станции метро',
    'Время до метро',
    'Адрес',
    'Площадь, м2',
    'Цена',
    'Залог',
    'Медианная_цена_x',
    'Медианная_цена_y',
    'Лифт',
    'Высота потолков, м',
    'Дополнительно',
    'Можно с детьми/животными',
    'Ремонт',
    'Балкон',
    'Окна',
    'Санузел',
    'Дом',
    'Парковка'
]

# Соответствующие английские названия колонок
new_names = [
    'ID',
    'number_of_rooms',
    'type',
    'subway_station_name',
    'walking_time_to_subway',
    'address',
    'area_m2',
    'price',
    'deposit',
    'median_price_x',
    'median_price_y',
    'lift',
    'ceiling_height_m',
    'additional_info',
    'allow_children_pets',
    'renovation',
    'balcony',
    'windows',
    'bathroom',
    'building',
    'parking'
]

# Создаем словарь соответствия и переименовываем колонки
name_map = dict(zip(current_names, new_names))
data.rename(columns=name_map, inplace=True)

In [412]:
data.drop(columns=['address', 'median_price_x', 'median_price_y'], inplace=True)

In [413]:
data.head(10)

,ID,number_of_rooms,type,subway_station_name,walking_time_to_subway,area_m2,price,deposit,lift,ceiling_height_m,additional_info,allow_children_pets,renovation,balcony,windows,bathroom,building,parking,Метро
0,273973191,3.0,Квартира,Смоленская,9 мин.,120.0,130000.0,130000.0,Пасс (1),3.0,"Мебель на кухне, Ванна, Стиральная машина, Кон...",Можно с животными,Евроремонт,Балкон (1),На улицу,Совмещенный (1),"5/10, Сталинский",открытая,9 мин.
1,273999490,3.0,Квартира,Смоленская,2 мин.,80.0,100000.0,110000.0,Пасс (1),2.5,"Мебель в комнатах, Мебель на кухне, Душевая ка...",Можно с детьми,Евроремонт,Балкон (3),На улицу и двор,Совмещенный (1),"4/8, Кирпичный",наземная,2 мин.
2,274995950,2.0,Квартира,Арбатская,10 мин.,30.0,120000.0,120000.0,Пасс (1),2.5,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Можно с детьми, Можно с животными",Евроремонт,Балкон (1),На улицу,Совмещенный (1),19/26,наземная,10 мин.
3,271265359,2.0,Квартира,Смоленская,5 мин.,60.0,90000.0,90000.0,Пасс (1),3.2,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Можно с детьми,Евроремонт,Балкон (1),На улицу и двор,Совмещенный (1),5/6,наземная,5 мин.
4,274223065,2.0,Квартира,Смоленская,8 мин.,60.0,120000.0,120000.0,Пасс (1),2.5,"Мебель в комнатах, Мебель на кухне, Душевая ка...",Можно с детьми,Дизайнерский,Балкон (1),На улицу,Совмещенный (1),3/6,наземная,8 мин.
5,274528249,1.0,Квартира,Арбатская,3 мин.,45.0,120000.0,120000.0,Пасс (1),2.5,"Мебель в комнатах, Мебель на кухне, Душевая ка...",Можно с детьми,Дизайнерский,Балкон (1),Во двор,Совмещенный (1),"6/7, Кирпичный",наземная,3 мин.
6,274431438,3.0,Квартира,Смоленская,3 мин.,65.0,130000.0,130000.0,Пасс (1),3.2,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Можно с детьми, Можно с животными",Евроремонт,Балкон (1),На улицу и двор,Совмещенный (1),7/7,наземная,3 мин.
7,274319903,3.0,Квартира,Смоленская,3 мин.,75.0,99000.0,99000.0,"Пасс (1), Груз (1)",2.5,"Мебель в комнатах, Мебель на кухне, Стиральная...","Можно с детьми, Можно с животными",Косметический,Балкон (1),Во двор,Раздельный (1),"9/10, Кирпичный",открытая,3 мин.
8,273427928,3.0,Квартира,Смоленская,11 мин.,87.0,100000.0,100000.0,Пасс (1),2.5,"Мебель в комнатах, Мебель на кухне, Стиральная...",Можно с детьми,Дизайнерский,Балкон (1),На улицу,Совмещенный (1),"3/7, Кирпичный",наземная,11 мин.
9,273763131,3.0,Квартира,Смоленская,4 мин.,91.0,95000.0,95000.0,Пасс (1),2.5,"Мебель в комнатах, Мебель на кухне, Стиральная...",Можно с детьми,Без ремонта,Балкон (1),Во двор,Раздельный (1),9/10,наземная,4 мин.


In [414]:
data.drop_duplicates('ID', inplace=True)

In [415]:
data.to_csv('Data')